In [ ]:
import numpy as np
import pandas as pd

## **1. Data Definition**

### Customers Data

In [ ]:
demands = [0, 5, 20, 10, 20, 85, 65, 30, 20, 70, 30]
customer_data = pd.DataFrame({'customer': range(1, 12), 'd': demands})
customer_data

### Vehicles Data

In [ ]:
num_vehicles = 6
vehicle_capacity = 100

### Routes Costs

In [ ]:
distance_matrix = np.array([
    [0, 13, 6, 55, 93, 164, 166, 168, 169, 231, 212],
    [13, 0, 11, 66, 261, 175, 177, 179, 180, 239, 208],
    [6, 11, 0, 60, 97, 168, 171, 173, 174, 239, 209],
    [55, 66, 60, 0, 82, 113, 115, 117, 117, 295, 265],
    [93, 261, 97, 82, 0, 113, 115, 117, 118, 333, 302],
    [164, 175, 168, 113, 113, 0, 6, 7, 2, 403, 374],
    [166, 177, 171, 115, 115, 6, 0, 8, 7, 406, 376],
    [168, 179, 173, 117, 117, 7, 8, 0, 3, 408, 378],
    [169, 180, 174, 117, 118, 2, 7, 3, 0, 409, 379],
    [231, 239, 239, 295, 333, 403, 406, 408, 409, 0, 46],
    [212, 208, 209, 265, 302, 374, 376, 378, 379, 46, 0]
])

distances = pd.DataFrame(distance_matrix, index=range(1, 12), columns=range(1, 12))
distances

## **2. Graph**

## **3. Genetic Algorithm**

## **4. Problem Representation**

## **5. Fitness Function**

## **6. Genetic Algorithm Execution**

## **7. Route Visualization**

## **8. Results Analysis**